## The data figure we want to reproduce (from [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3039263/figure/fig01/))

![data-figure](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3039263/bin/tjp0589-0101-f1.jpg)

In [6]:
%matplotlib inline

# Built-in libs
import os
import sys
import warnings

# Common libs
import matplotlib.pyplot as plt
import quantities as pq

# Open Worm libs
import owtests
import c302

# Testing libs
warnings.simplefilter('ignore')
import sciunit
from neuronunit.tests import APWidthTest,APAmplitudeTest
from c302.muscleunit import vm_plot,models as sciunit_models
!pwd

/home/jovyan/work/openworm/tests/owtests


In [2]:
# Create a MuscleModel instance with a particular configuration
# This model has a neuron and a muscle cell
model = sciunit_models.MuscleModel('IClampBWM', 'C2', 1000, 0.05, config_package="notebooks.configs")

Set default parameters for C
Set default parameters for C2
Opened file: /home/jovyan/work/openworm/CElegansNeuroML/CElegans/pythonScripts/c302/../../../herm_full_edgelist.csv
Opened file: /home/jovyan/work/openworm/CElegansNeuroML/CElegans/pythonScripts/c302/../../../herm_full_edgelist.csv
c302      >>>  Positioning muscle: MDR01 at (80,-270,80)
c302      >>>  Writing generated network to: /home/jovyan/work/openworm/CElegansNeuroML/CElegans/pythonScripts/c302/examples/c302_C2_IClampBWM.nml
(Re)written network file to: /home/jovyan/work/openworm/CElegansNeuroML/CElegans/pythonScripts/c302/examples/c302_C2_IClampBWM.nml


In [3]:
# Dictionary of observations, in this case two ephys properties from one paper
from neuronunit.tests import dynamics 
import quantities as pq

doi = 'doi:10.1113/jphysiol.2010.200683'
observations={doi:{'ap_amplitude':{'mean':45.1*pq.mV,
                                   'sem':0.7*pq.mV,
                                   'n':25},
                   'ap_width':{'mean':19.7*pq.ms,
                               'sem':1.0*pq.ms,
                               'n':25}}}    

# Instantiate two tests based on these properties
ap_width_test = APWidthTest(observation=observations[doi]['ap_width'])
ap_amplitude_test = APAmplitudeTest(observation=observations[doi]['ap_amplitude'])



observation = {}
averaged:  149.0718879752502

observation[doi] = {}
observation[doi]['isi'] = 598.0*pq.ms
observation[doi]['mean'] = 598.0*pq.ms
observation[doi]['std'] = 37.0*pq.ms
isi_test = dynamics.ISITest(observation=observation[doi])
# Put them together in a test suite

# Note these values where derived from the adjacent notebook test_data_CV_histogram
#0.4486588231066874 0.10415658416895163

observation = {}
observation[doi] = {}
observation[doi]['sps'] = 0.4486588231066874*pq.dimensionless
observation[doi]['mean'] = 0.4486588231066874*pq.dimensionless
observation[doi]['std'] = 0.10415658416895163*pq.dimensionless
cv_test = dynamics.CVTest(observation=observation[doi])

cv_test = dynamics.CVTest(observation = observation[doi])

ap_tests = sciunit.TestSuite([ap_width_test,ap_amplitude_test,isi_test,cv_test], name="AP CV and ISI Tests")
# Put them together in a test suite
ap_tests = sciunit.TestSuite([ap_width_test,ap_amplitude_test,isi_test,cv_test], name="AP ISI and CV Tests")

#ap_tests = sciunit.TestSuite([ap_width_test,ap_amplitude_test,isi_test], name="AP and ISI Tests")

In [5]:
# Judge the membrane potential of the muscle 
# One could judge the neuron instead with appropriate keyword parameters, see below)
%time score_matrix = ap_tests.judge(model) 
import pandas as pd
score_matrix.style.set_properties(**{'font-size':'16pt'})

score = cv_test.judge(model) 

print('prediction',score.prediction['mean'])
print('observation',score.observation['mean'])
score = isi_test.judge(model) 
print('prediction',score.prediction)
print('observation',score.observation)


pyNeuroML >>> Reloading data specified in LEMS file: /tmp/tmpr6louqgs/LEMS_c302_C2_IClampBWM.xml (/tmp/tmpr6louqgs/LEMS_c302_C2_IClampBWM.xml), base_dir: /tmp/tmp3tkluem1, cwd: /home/jovyan/work/openworm/tests/owtests


{'isi': array(598.0) * ms, 'mean': array(598.0) * ms, 'std': array(37.0) * ms} {'isi': array(270.87142857142857) * ms}


CPU times: user 304 ms, sys: 16 ms, total: 320 ms
Wall time: 311 ms
prediction 0.4946201516295575 dimensionless
observation 0.4486588231066874 dimensionless
{'isi': array(598.0) * ms, 'mean': array(598.0) * ms, 'std': array(37.0) * ms} {'isi': array(270.87142857142857) * ms}
prediction {'isi': array(270.87142857142857) * ms}
observation {'isi': array(598.0) * ms, 'mean': array(598.0) * ms, 'std': array(37.0) * ms}


/opt/conda/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)


,APWidthTest,APAmplitudeTest,ISITest,CVTest
LEMS_c302_C2_IClampBWM,Z = -3.06,Z = -11.03,Ratio = 0.45,Ratio = 2.35


In [ ]:
# Plot the model output for one of the tests
import matplotlib as mpl
mpl.rcParams.update({'font.size':18})
plt.figure(figsize=(12,6))
score_matrix['APWidthTest']['LEMS_c302_C2_IClampBWM'].plot_vm()
plt.savefig('model.pdf')

## Model methods could also be run directly as follows

In [ ]:
# These will use the cached results and so be much faster than the original run
%time neuron_vm = model.get_membrane_potential_neuron()
%time muscle_vm = model.get_membrane_potential_muscle()

In [ ]:
# Another way to plot the model output, in this case by cell type
vm_plot(muscle_vm,"Muscle")

In [ ]:
# Same as above, but for a different cell type in this model
# Note that in this flavor of the model, the neuron isn't doing anything at all, 
# and the muscle cell is firing spontaneously
vm_plot(neuron_vm,"Neuron")